In [19]:
import pandas as pd

In [20]:
inpath = "data/epitope-predictions/mhcnuggets.csv"

df = pd.read_csv(inpath)

display(df)

,allele,ic50,name,peptide,pos,rank
0,HLA-A01:01,2933.23,YP_173242.1,FSDGQGVPIAF,78,1.0
1,HLA-A01:01,5287.00,YP_173242.1,MADEIANLVLA,230,2.0
2,HLA-A01:01,5404.65,YP_173242.1,YSWFSGITQFQ,61,3.0
3,HLA-A01:01,5405.67,YP_173242.1,ELAPTPGAFFF,312,4.0
4,HLA-A01:01,5409.25,YP_173242.1,SFKTADGQQKQ,106,5.0
...,...,...,...,...,...,...
52619,HLA-B08:01,38784.20,YP_009724397.2,YKTFPPTEPKK,359,405.0
52620,HLA-B08:01,38856.74,YP_009724397.2,GGPSDSTGSNQ,17,406.0
52621,HLA-B08:01,38907.21,YP_009724397.2,EVTPSGTWLTY,322,407.0
52622,HLA-B08:01,38954.47,YP_009724397.2,TNSSPDDQIGY,75,408.0


In [22]:
df.groupby('name').size()

name
NP_073556.1        9854
YP_003771.1        9542
YP_009555245.1    11388
YP_009724397.2    10634
YP_173242.1       11206
dtype: int64

In [38]:
# Start with one MHC allele and later try to generalise
allele = "HLA-A01:01"

df = df[df['allele']==allele]

In [53]:
# Get the n peptides with the smallest ic50 for each name
n = 2
df.groupby(['name'])[['peptide','ic50']].apply(lambda x: x.nsmallest(n, columns=['ic50']))

peptide     ic50
name                                      
NP_073556.1    32136  RVDLSPKLHFY   644.16
               32137  NSEPEIPHFNQ  5290.11
YP_003771.1    22594  RVDLPPKVHFY   633.02
               22595  VGDNVQITYTY   921.54
YP_009555245.1 11206  GTDIDGVYWVA  5177.95
               11207  TSDPQFPILAE  5315.95
YP_009724397.2 41990  NSSPDDQIGYY   620.51
               41991  LLDRLNQLESK  5151.36
YP_173242.1    0      FSDGQGVPIAF  2933.23
               1      MADEIANLVLA  5287.00